In [1]:
import pandas as pd

In [2]:
train_rides_df = pd.read_csv('DBtrainrides.csv')

In [3]:
train_rides_df.head()

,ID,line,path,eva_nr,category,station,state,city,zip,long,lat,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info,arrival_delay_check,departure_delay_check
0,1573967790757085557-2407072312-14,20,Stolberg(Rheinl)Hbf Gl.44|Eschweiler-St.Jöris|...,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.767800,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time
1,349781417030375472-2407080017-1,18,NaN,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.767800,NaN,2024-07-08 00:17:00,NaN,NaN,0,0,NaN,on_time,on_time
2,7157250219775883918-2407072120-25,1,Hamm(Westf)Hbf|Kamen|Kamen-Methler|Dortmund-Ku...,8000406,4,Aachen-Rothe Erde,Nordrhein-Westfalen,Aachen,52066,6.116475,50.770202,2024-07-08 00:03:00,2024-07-08 00:04:00,2024-07-08 00:03:00,2024-07-08 00:04:00,0,0,NaN,on_time,on_time
3,349781417030375472-2407080017-2,18,Aachen Hbf,8000404,5,Aachen West,Nordrhein-Westfalen,Aachen,52072,6.070715,50.780360,2024-07-08 00:20:00,2024-07-08 00:21:00,NaN,NaN,0,0,NaN,on_time,on_time
4,1983158592123451570-2407080010-3,33,Herzogenrath|Kohlscheid,8000404,5,Aachen West,Nordrhein-Westfalen,Aachen,52072,6.070715,50.780360,2024-07-08 00:20:00,2024-07-08 00:21:00,2024-07-08 00:20:00,2024-07-08 00:21:00,0,0,NaN,on_time,on_time


In [4]:
zip_df=train_rides_df.groupby("zip").size().reset_index(name="count")
zip_df.head()

,zip,count
0,1067,2458
1,1069,2045
2,1097,3305
3,1109,1800
4,1127,597


In [5]:
train_rides_df.count()

ID                       2061357
line                     2061357
path                     1850002
eva_nr                   2061357
category                 2061357
station                  2061357
state                    2061357
city                     2061357
zip                      2061357
long                     2061357
lat                      2061357
arrival_plan             1850002
departure_plan           2061357
arrival_change           1585727
departure_change         1721431
arrival_delay_m          2061357
departure_delay_m        2061357
info                      645341
arrival_delay_check      2061357
departure_delay_check    2061357
dtype: int64

2,061,357–1,850,002 = 211,355 fewer cases in the 'path' column

In [6]:
# Step 2: Create a new DataFrame with a count of how often each 'eva_nr' is mentioned
station_mapping_df = train_rides_df.groupby(['eva_nr', 'station', 'category', 'city', 'long', 'lat']).size().reset_index(name='count')

# Step 3: Optionally, reset the index for a cleaner DataFrame
station_mapping_df = station_mapping_df.reset_index(drop=True)

# Display the new DataFrame
station_mapping_df.head()

,eva_nr,station,category,city,long,lat,count
0,8000001,Aachen Hbf,2,Aachen,6.091499,50.767800,1488
1,8000002,Aalen Hbf,3,Aalen,10.096271,48.841013,823
2,8000004,Altenbeken,3,Altenbeken,8.943319,51.766433,848
3,8000007,Alzey,3,Alzey,8.109749,49.750200,591
4,8000009,Ansbach,3,Ansbach,10.578239,49.298032,829


In [7]:
station_mapping_df.count()

eva_nr      1996
station     1996
category    1996
city        1996
long        1996
lat         1996
count       1996
dtype: int64

what is the average number of stations per category? go check the 'category' column

In [8]:
# # Step 1: Split the 'path' into individual stations where 'path' is not empty
# # Create a new DataFrame that splits the 'path' column
# stations_df = train_rides_df['path'].str.split('|', expand=True)
# 
# # Step 2: Keep the same 'eva_nr', 'station', and 'city' columns, add the split station columns
# split_stations_df = pd.concat([train_rides_df[['ID','eva_nr', 'station', 'city']], stations_df], axis=1)
# 
# split_stations_df.head()

In [9]:
# Step 1: Split the 'path' into individual stations where 'path' is not empty
train_rides_df['path'] = train_rides_df['path'].fillna('')

# Step 2: Split each 'path' entry by '|', creating multiple rows per station
split_stations_df = train_rides_df.assign(current_station=train_rides_df['path'].str.split('|')).explode('current_station').drop(columns=['path'])

# Display the result
split_stations_df.head()

,ID,line,eva_nr,category,station,state,city,zip,long,lat,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info,arrival_delay_check,departure_delay_check,current_station
0,1573967790757085557-2407072312-14,20,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.7678,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Stolberg(Rheinl)Hbf Gl.44
0,1573967790757085557-2407072312-14,20,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.7678,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Eschweiler-St.Jöris
0,1573967790757085557-2407072312-14,20,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.7678,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf Poststraße
0,1573967790757085557-2407072312-14,20,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.7678,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf-Mariadorf
0,1573967790757085557-2407072312-14,20,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.7678,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf-Kellersberg


In [10]:
split_stations_df.drop(['category', 'station', 'state', 'city', 'zip', 'long', 'lat'], axis=1, inplace=True)
split_stations_df.head()

,ID,line,eva_nr,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info,arrival_delay_check,departure_delay_check,current_station
0,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Stolberg(Rheinl)Hbf Gl.44
0,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Eschweiler-St.Jöris
0,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf Poststraße
0,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf-Mariadorf
0,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf-Kellersberg


In [11]:
# Perform the merge operation
# First, ensure the station names match by stripping any extra whitespace
station_mapping_df['station'] = station_mapping_df['station'].str.strip()
split_stations_df['current_station'] = split_stations_df['current_station'].str.strip()

# Merge the DataFrames on the station name (current_station and station)
split_stations_df_with_station_mapping_df = split_stations_df.merge(station_mapping_df, how='left', left_on='current_station', right_on='station')

In [12]:
split_stations_df_with_station_mapping_df

,ID,line,eva_nr_x,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info,arrival_delay_check,departure_delay_check,current_station,eva_nr_y,station,category,city,long,lat,count
0,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Stolberg(Rheinl)Hbf Gl.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Eschweiler-St.Jöris,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf Poststraße,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf-Mariadorf,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf-Kellersberg,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19714102,3370285438001482281-2407142234-7,8,8003775,2024-07-14 23:10:00,2024-07-14 23:11:00,2024-07-14 23:11:00,2024-07-14 23:12:00,1,1,Information,on_time,on_time,Lübeck Hbf,8000237.0,Lübeck Hbf,2.0,Lübeck,10.669821,53.867547,1428.0
19714103,-8774053210575864323-2407142305-3,80,8003775,2024-07-14 23:17:00,2024-07-14 23:18:00,2024-07-14 23:17:00,2024-07-14 23:18:00,0,0,Information,on_time,on_time,Bad Oldesloe,8000023.0,Bad Oldesloe,2.0,Bad Oldesloe,10.382541,53.805390,800.0
19714104,-8774053210575864323-2407142305-3,80,8003775,2024-07-14 23:17:00,2024-07-14 23:18:00,2024-07-14 23:17:00,2024-07-14 23:18:00,0,0,Information,on_time,on_time,Reinfeld(Holst),NaN,NaN,NaN,NaN,NaN,NaN,NaN
19714105,-1537118689903044118-2407142354-1,11,8001580,NaN,2024-07-14 23:54:00,NaN,NaN,0,0,Information. (Quelle: zuginfo.nrw),on_time,on_time,,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
station_mapping_df

,eva_nr,station,category,city,long,lat,count
0,8000001,Aachen Hbf,2,Aachen,6.091499,50.767800,1488
1,8000002,Aalen Hbf,3,Aalen,10.096271,48.841013,823
2,8000004,Altenbeken,3,Altenbeken,8.943319,51.766433,848
3,8000007,Alzey,3,Alzey,8.109749,49.750200,591
4,8000009,Ansbach,3,Ansbach,10.578239,49.298032,829
...,...,...,...,...,...,...,...
1991,8089472,Lichterfelde Süd,5,Berlin,13.309040,52.410899,1544
1992,8089473,Osdorferstraße,5,Berlin,13.314288,52.418841,1536
1993,8089474,Schöneberg,4,Berlin,13.352068,52.479377,5831
1994,8089537,Julius-Leber-Brücke,5,Berlin,13.361266,52.486519,2191


Visualise the stations in a map (/grid) (without it being mapped in a real map) and see if there are any patterns. George \\

Additionally, use the same map to see the delay (in % of total rides) per station and per zip. Delay in this case would be if the train did not arrive / leave at the intended time. George \\


See if we can use K-means to cluster the stations into mini regions to see if the entire region has delay problems. We do not know what k to use, so we can try a few values. Check the distance per stations and see by eye if the clusters make sense to create. Raphael

In [20]:
split_stations_df_with_station_mapping_df.head(10)

,ID,line,eva_nr_x,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info,arrival_delay_check,departure_delay_check,current_station,eva_nr_y,station,category,city,long,lat,count
0,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Stolberg(Rheinl)Hbf Gl.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Eschweiler-St.Jöris,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf Poststraße,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf-Mariadorf,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf-Kellersberg,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf-Annapark,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf-Busch,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Herzogenrath-August-Schmidt-Platz,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Herzogenrath-Alt-Merkstein,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Herzogenrath,8002806.0,Herzogenrath,3.0,Herzogenrath,6.094486,50.870916,1899.0


In [21]:
# Filter rows that have latitude and longitude values
split_stations_df_with_station_mapping_df_with_coords = split_stations_df_with_station_mapping_df.dropna(subset=['long', 'lat'])

# Create a base map
m = folium.Map(location=[split_stations_df_with_station_mapping_df_with_coords['lat'].mean(), split_stations_df_with_station_mapping_df_with_coords['long'].mean()], zoom_start=12)

# Group by ID and plot the stations and lines
for unique_id, group in split_stations_df_with_station_mapping_df_with_coords.groupby('ID'):
    # Extract the coordinates for the stations with the same ID
    coordinates = list(zip(group['lat'], group['long']))
    
    # Plot each station
    for _, row in group.iterrows():
        folium.Marker(
            location=[row['lat'], row['long']],
            popup=row['current_station'],
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(m)
    
    # Connect the stations with a line
    folium.PolyLine(coordinates, color='blue', weight=2.5, opacity=0.8).add_to(m)

# Save map to an HTML file
m.save('stations_map.html')

# Display the map (if running in a Jupyter environment, this will display the map)
m

NameError: name 'folium' is not defined

In [14]:
# # Step 1: Split the 'path' column into separate station columns for each stop
# station_columns_df = train_rides_df['path'].str.split('|', expand=True)
# 
# # Step 2: Concatenate with relevant identifiers and location details to keep track of each train ride
# train_rides_with_stations_df = pd.concat([train_rides_df[['ID', 'eva_nr', 'station', 'city']], station_columns_df], axis=1)
# 
# # Step 3: Reshape to a long format with each station as a separate row, preserving ride details
# stations_long_format_df = train_rides_with_stations_df.melt(
#     id_vars=['ID', 'eva_nr', 'station', 'city'], 
#     var_name='station_sequence', 
#     value_name='station_name'
# )
# stations_long_format_df = stations_long_format_df.dropna(subset=['station_name'])

In [15]:
# # Step 4: Merge with 'station_mapping_df' to retrieve 'eva_nr' for each station
# stations_with_eva_df = stations_long_format_df.merge(
#     station_mapping_df[['station', 'eva_nr']], 
#     left_on='station_name',
#     right_on='station',
#     how='left',
#     suffixes=('', '_mapped')
# )
# 
# # Step 5: Drop duplicate or unnecessary columns for a clean DataFrame
# final_station_mapping_df = stations_with_eva_df.drop(columns=['station'])
# 
# # Step 6: Reset index for a cleaner, ordered display
# final_station_mapping_df = final_station_mapping_df.reset_index(drop=True)
# 
# # Display the final DataFrame
# final_station_mapping_df

In [16]:
import plotly.express as px

In [17]:
# # Generate a blank point graph
# fig_blank = px.scatter(train_rides_df, x="long", y="lat", title="Blank Point Graph of Longitude and Latitude")
# 
# # Show plot
# fig_blank.show()

In [18]:
# # Generate a map using longitude and latitude
# fig_map = px.scatter_mapbox(train_rides_df, lat="lat", lon="long", 
#                             mapbox_style="open-street-map", 
#                             title="Map of Train Stations Based on Longitude and Latitude")
# # Show plot
# fig_map.show()